In [22]:
import pandas as pd
import numpy as np
from typing import Dict
from datetime import datetime
import os

In [23]:
def compare_dataframes(df1: pd.DataFrame, df2: pd.DataFrame, df1_name: str = "DataFrame 1", 
                      df2_name: str = "DataFrame 2", verbose: bool = True) -> Dict:
    """
    Compare columns between two DataFrames and identify differences.
    
    Parameters:
    -----------
    df1 : pandas.DataFrame
        First DataFrame to compare
    df2 : pandas.DataFrame
        Second DataFrame to compare
    df1_name : str, default="DataFrame 1"
        Name of the first DataFrame for output
    df2_name : str, default="DataFrame 2"
        Name of the second DataFrame for output
    verbose : bool, default=True
        If True, prints detailed comparison results
        
    Returns:
    --------
    Dict containing:
        - columns_only_in_df1: List of columns unique to df1
        - columns_only_in_df2: List of columns unique to df2
        - common_columns: List of columns present in both DataFrames
        - dtype_differences: Dict of columns with different dtypes
    """
    
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)
    
    only_in_df1 = sorted(list(cols1 - cols2))
    only_in_df2 = sorted(list(cols2 - cols1))
    common_cols = sorted(list(cols1 & cols2))
    
    dtype_diff = {}
    for col in common_cols:
        if df1[col].dtype != df2[col].dtype:
            dtype_diff[col] = {
                'df1_dtype': str(df1[col].dtype),
                'df2_dtype': str(df2[col].dtype)
            }
    
    result = {
        'columns_only_in_df1': only_in_df1,
        'columns_only_in_df2': only_in_df2,
        'common_columns': common_cols,
        'dtype_differences': dtype_diff
    }
    
    if verbose:
        print("\nDataFrame Column Comparison Results:")
        print("-" * 35)
        
        print(f"\nColumns only in {df1_name} ({len(only_in_df1)}):")
        for col in only_in_df1:
            print(f"  - {col}")
            
        print(f"\nColumns only in {df2_name} ({len(only_in_df2)}):")
        for col in only_in_df2:
            print(f"  - {col}")
            
        print(f"\nCommon columns ({len(common_cols)}):")
        for col in common_cols:
            print(f"  - {col}")
            
        if dtype_diff:
            print("\nColumns with different dtypes:")
            for col, dtypes in dtype_diff.items():
                print(f"  - {col}:")
                print(f"      {df1_name}: {dtypes['df1_dtype']}")
                print(f"      {df2_name}: {dtypes['df2_dtype']}")
    
    return result



def merge_datasets(baseline_df: pd.DataFrame, final_df: pd.DataFrame, 
                  merge_keys: list = ['N_sans_P'], validate_dtypes: bool = True) -> pd.DataFrame:
    """
    Merge Baseline and Final datasets using composite keys with validation and handling of duplicate columns.
    
    Parameters:
    -----------
    baseline_df : pandas.DataFrame
        Baseline DataFrame
    final_df : pandas.DataFrame
        Final DataFrame
    merge_keys : list, default=['NEFG2', 'LT_date']
        List of columns to use as merge keys
    validate_dtypes : bool, default=True
        Whether to validate and harmonize dtypes before merging
        
    Returns:
    --------
    pandas.DataFrame
        Merged DataFrame
    """
    baseline = baseline_df.copy()
    final = final_df.copy()
    
    for key in merge_keys:
        if key not in baseline.columns or key not in final.columns:
            raise ValueError(f"Merge key '{key}' must exist in both DataFrames")
    
    common_cols = [col for col in baseline.columns.intersection(final.columns) 
                  if col not in merge_keys]
    
    if validate_dtypes:
        for col in common_cols:
            if pd.api.types.is_numeric_dtype(baseline[col]) and \
               pd.api.types.is_numeric_dtype(final[col]):
                baseline[col] = baseline[col].astype(float)
                final[col] = final[col].astype(float)
    
    rename_dict = {col: f"{col}_final" for col in common_cols}
    final = final.rename(columns=rename_dict)
    
    merged_df = pd.merge(baseline, final, on=merge_keys, how='outer', 
                        validate=None, indicator=True)
    
    merged_df['source'] = merged_df['_merge'].map({
        'left_only': 'baseline_only',
        'right_only': 'final_only',
        'both': 'both'
    })
    
    print("\nMerge Summary:")
    print("-" * 20)
    print(f"Merge keys used: {', '.join(merge_keys)}")
    merge_counts = merged_df['source'].value_counts()
    print(f"Total rows in merged dataset: {len(merged_df)}")
    print(f"Rows from baseline only: {merge_counts.get('baseline_only', 0)}")
    print(f"Rows from final only: {merge_counts.get('final_only', 0)}")
    print(f"Rows in both datasets: {merge_counts.get('both', 0)}")
    
    for key in merge_keys:
        null_counts = merged_df[key].isna().sum()
        if null_counts > 0:
            print(f"\nWarning: {null_counts} null values found in merge key '{key}'")
        
        if key == 'NEFG2':
            # Check for potential formatting differences in NEFG2
            print(f"\nNEFG2 unique values sample in baseline:")
            print(baseline[key].value_counts().head())
            print(f"\nNEFG2 unique values sample in final:")
            print(final[key].value_counts().head())
    
    merged_df = merged_df.drop('_merge', axis=1)
    
    for col in common_cols:
        final_col = f"{col}_final"
        if final_col in merged_df.columns:
            merged_df[f"{col}_combined"] = np.where(
                merged_df[final_col].notna(),
                merged_df[final_col],
                merged_df[col]
            )

    
    return merged_df



def analyze_merge_results(baseline_df: pd.DataFrame, final_df: pd.DataFrame, 
                         merged_df: pd.DataFrame, merge_key: str = 'LT_date', 
                         date_format: str = None) -> dict:
    """
    Analyze the results of merging baseline and final datasets, focusing on unmatched cases.
    
    Parameters:
    -----------
    baseline_df : pandas.DataFrame
        Original baseline DataFrame
    final_df : pandas.DataFrame
        Original final DataFrame
    merged_df : pandas.DataFrame
        Merged DataFrame containing 'source' column
    merge_key : str, default='LT_date'
        Column used as merge key
    date_format : str, optional
        Expected date format if merge_key is a date
        
    Returns:
    --------
    dict
        Dictionary containing analysis results
    """
    
    def check_date_formatting(df, col):
        """Check for potential date formatting issues"""
        try:
            dates = pd.to_datetime(df[col])
            return {
                'min_date': dates.min(),
                'max_date': dates.max(),
                'null_dates': dates.isna().sum(),
                'unique_formats': df[col].astype(str).drop_duplicates().tolist()[:5]
            }
        except:
            return "Date conversion failed"

    baseline_only = merged_df[merged_df['source'] == 'baseline_only']
    final_only = merged_df[merged_df['source'] == 'final_only']
    matched = merged_df[merged_df['source'] == 'both']
    
    date_analysis = {
        'baseline': check_date_formatting(baseline_df, merge_key),
        'final': check_date_formatting(final_df, merge_key)
    }
    
    numeric_analysis = {}
    for df_name, df in [('baseline_only', baseline_only), 
                       ('final_only', final_only), 
                       ('matched', matched)]:
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        numeric_analysis[df_name] = {
            col: {
                'mean': df[col].mean(),
                'median': df[col].median(),
                'std': df[col].std(),
                'null_count': df[col].isna().sum(),
                'null_pct': (df[col].isna().sum() / len(df)) * 100
            } for col in numeric_cols
        }
    
    print("\nDetailed Merge Analysis")
    print("=" * 50)
    
    print("\n1. Basic Statistics:")
    print(f"Total records in merged dataset: {len(merged_df)}")
    print(f"Matched records: {len(matched)} ({len(matched)/len(merged_df)*100:.1f}%)")
    print(f"Unmatched from baseline: {len(baseline_only)} ({len(baseline_only)/len(merged_df)*100:.1f}%)")
    print(f"Unmatched from final: {len(final_only)} ({len(final_only)/len(merged_df)*100:.1f}%)")
    
    if isinstance(date_analysis['baseline'], dict):
        print("\n2. Date Range Analysis:")
        print("\nBaseline Dataset:")
        print(f"Date range: {date_analysis['baseline']['min_date']} to {date_analysis['baseline']['max_date']}")
        print(f"Null dates: {date_analysis['baseline']['null_dates']}")
        print("\nFinal Dataset:")
        print(f"Date range: {date_analysis['final']['min_date']} to {date_analysis['final']['max_date']}")
        print(f"Null dates: {date_analysis['final']['null_dates']}")
    
    print("\n3. Unmatched Cases Analysis:")
    print("\nBaseline-only cases:")
    print(baseline_only[merge_key].to_string())
    print("\nFinal-only cases:")
    print(final_only[merge_key].to_string())
    
    print("\n4. Common Columns Value Distribution:")
    common_cols = set(baseline_df.columns) & set(final_df.columns)
    for col in common_cols:
        if col in numeric_analysis['matched']:
            print(f"\nColumn: {col}")
            print("Mean values:")
            print(f"  Matched cases: {numeric_analysis['matched'][col]['mean']:.2f}")
            if col in numeric_analysis['baseline_only']:
                print(f"  Baseline-only: {numeric_analysis['baseline_only'][col]['mean']:.2f}")
            if col in numeric_analysis['final_only']:
                print(f"  Final-only: {numeric_analysis['final_only'][col]['mean']:.2f}")
    
    return {
        'date_analysis': date_analysis,
        'numeric_analysis': numeric_analysis,
        'unmatched_baseline': baseline_only[merge_key].tolist(),
        'unmatched_final': final_only[merge_key].tolist(),
        'match_summary': {
            'total': len(merged_df),
            'matched': len(matched),
            'unmatched_baseline': len(baseline_only),
            'unmatched_final': len(final_only)
        }
    }

def investigate_unmatched_cases(baseline_df, final_df, merged_df):
    """
    Wrapper function to analyze merge results and provide recommendations
    """
    results = analyze_merge_results(baseline_df, final_df, merged_df)
    
    print("\nRecommendations:")
    print("=" * 50)
    
    if isinstance(results['date_analysis']['baseline'], dict):
        baseline_range = (results['date_analysis']['baseline']['min_date'],
                         results['date_analysis']['baseline']['max_date'])
        final_range = (results['date_analysis']['final']['min_date'],
                      results['date_analysis']['final']['max_date'])
        
        if baseline_range != final_range:
            print("\n1. Date Range Mismatch Detected:")
            print("Consider investigating cases outside the overlapping date range")
    
    if len(results['unmatched_baseline']) > 0:
        print("\n2. Baseline Unmatched Cases Pattern:")
        print("Review these specific dates for potential data entry issues")
    
    if len(results['unmatched_final']) > 0:
        print("\n3. Final Unmatched Cases Pattern:")
        print("Review these specific dates for potential data entry issues")
    
    print("\nSpecific Actions:")
    print("1. Verify date formats are consistent between datasets")
    print("2. Check for any duplicate entries in either dataset")
    print("3. Investigate any significant value differences in common columns")
    print("4. Consider if unmatched cases should be excluded or require additional data collection")
    
    return results


def save_merged_dataset(merged_df, output_dir="./data", 
                       prefix="merged", include_timestamp=True):
    """
    Save the merged dataset with appropriate documentation
    
    Parameters:
    -----------
    merged_df : pandas.DataFrame
        The merged dataframe to save
    output_dir : str, default="./data"
        Directory where to save the files
    prefix : str, default="merged"
        Prefix for the output files
    include_timestamp : bool, default=True
        Whether to include timestamp in filenames
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S") if include_timestamp else ""
    base_filename = f"{prefix}_{timestamp}" if timestamp else prefix
    csv_path = os.path.join(output_dir, f"{base_filename}.csv")
    merged_df.to_csv(csv_path, index=False)
    
    metadata = {
        'total_rows': len(merged_df),
        'total_columns': len(merged_df.columns),
        'columns_info': {col: str(dtype) for col, dtype in merged_df.dtypes.items()},
        'missing_values': merged_df.isna().sum().to_dict(),
        'source_distribution': merged_df['source'].value_counts().to_dict() if 'source' in merged_df.columns else None,
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    }
    
    metadata_path = os.path.join(output_dir, f"{base_filename}_metadata.csv")
    pd.DataFrame([metadata]).to_csv(metadata_path, index=False)
    
    print("\nMerged dataset saved successfully!")
    print(f"Main dataset: {csv_path}")
    print("Metadata: {metadata_path}")
    print("\nDataset summary:")
    print(f"- Total rows: {metadata['total_rows']:,}")
    print(f"- Total columns: {metadata['total_columns']}")
    if metadata['source_distribution']:
        print("\nSource distribution:")
        for source, count in metadata['source_distribution'].items():
            print(f"- {source}: {count:,} rows")
    
    return {'data_path': csv_path, 'metadata_path': metadata_path, 'metadata': metadata}


In [24]:
baseline_df = pd.read_csv("../data/original/base_hcpc_baseline.csv", sep=";")
final_df = pd.read_csv("../data/original/base_hcpc_final.csv", sep=";")
differences = compare_dataframes(baseline_df, final_df, df1_name="Baseline", df2_name="Final")


DataFrame Column Comparison Results:
-----------------------------------

Columns only in Baseline (52):
  - CLAD_ddn
  - CLAD_delay
  - CLAD_delay_2
  - CLAD_status
  - CMV_DR
  - DPG3_72h
  - DSA_FINAL
  - DSA_FINAL_noNA
  - DSA_bin_1000
  - DSA_bin_3000
  - DSA_bin_500
  - DSA_last
  - DSA_last_date
  - DSA_pre
  - ECMO_postop
  - ECMO_postop_duree_jours
  - LAS
  - LAS_2
  - LAS_SU
  - LT_age
  - LT_age_2
  - NEFG
  - SU
  - aetiology
  - aetiology_2
  - aetiology_3
  - aetiology_3_and_1
  - birth_date
  - bmi
  - center
  - donneur_ISHLT
  - donneur_PF
  - donneur_age
  - donneur_tabac
  - donneur_tabac_2
  - gender
  - gender_2
  - graft_survival_ddn
  - graft_survival_delay
  - graft_survival_delay_2
  - graft_survival_status
  - induction
  - ischemie_duree
  - natt
  - nom_prenom_match
  - patient_survival_ddn
  - patient_survival_delay
  - patient_survival_delay_2
  - patient_survival_status
  - re_tx
  - taille
  - type_tp

Columns only in Final (9):
  - date_prelevement
  

In [25]:
merged_df = merge_datasets(baseline_df, final_df)


Merge Summary:
--------------------
Merge keys used: N_sans_P
Total rows in merged dataset: 5038
Rows from baseline only: 2519
Rows from final only: 2519
Rows in both datasets: 0


/var/folders/4l/60lgrf9s6sqbf6lcr70gbl2c0000gn/T/ipykernel_68705/940591594.py:118: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  merged_df = pd.merge(baseline, final, on=merge_keys, how='outer',


In [26]:
analysis_results = investigate_unmatched_cases(baseline_df, final_df, merged_df)


Detailed Merge Analysis

1. Basic Statistics:
Total records in merged dataset: 5038
Matched records: 0 (0.0%)
Unmatched from baseline: 2519 (50.0%)
Unmatched from final: 2519 (50.0%)

2. Date Range Analysis:

Baseline Dataset:
Date range: 2010-02-16 00:00:00 to 2019-11-01 00:00:00
Null dates: 0

Final Dataset:
Date range: 2010-02-16 00:00:00 to 2019-11-01 00:00:00
Null dates: 0

3. Unmatched Cases Analysis:

Baseline-only cases:
0       2015-07-31
1       2015-07-31
2       2015-07-31
3       2015-07-31
4       2015-07-31
5       2015-07-31
6       2015-07-31
7       2012-10-08
8       2012-10-08
9       2012-10-08
10      2012-10-08
35      2012-10-08
317     2012-10-08
632     2012-10-08
968     2012-10-08
1330    2012-10-08
1744    2013-03-14
1810    2013-03-14
2163    2013-03-14
2538    2013-03-14
2539    2013-03-14
2540    2013-03-14
2541    2013-03-14
2542    2013-03-14
2543    2015-02-10
2544    2015-02-10
2545    2015-02-10
2546    2015-02-10
2547    2015-02-10
2548    2015-02

/Users/sarrabenyahia/Documents/GitHub/longitudinal_BTB_clustering/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/sarrabenyahia/Documents/GitHub/longitudinal_BTB_clustering/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/sarrabenyahia/Documents/GitHub/longitudinal_BTB_clustering/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/sarrabenyahia/Documents/GitHub/longitudinal_BTB_clustering/venv/lib/python3.11/site-packages/numpy/lib/_nanfunctions_impl.py:1241: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/sarrabenyahia/Documents/GitHub/longitudinal_BTB_clustering/venv/lib/p

In [27]:
save_info = save_merged_dataset(merged_df, 
                              output_dir="../data/merged",
                              prefix="baseline_final_merged")


Merged dataset saved successfully!
Main dataset: ../data/merged/baseline_final_merged_20241029_115244.csv
Metadata: {metadata_path}

Dataset summary:
- Total rows: 5,038
- Total columns: 93

Source distribution:
- baseline_only: 2,519 rows
- final_only: 2,519 rows
- both: 0 rows


In [28]:
def compare_duplicate_columns(merged_df, original_col_names):
    """
    Compare values in columns that were duplicated during merge with improved string comparison
    
    Parameters:
    -----------
    merged_df : pandas.DataFrame
        The merged dataframe containing duplicate columns with suffixes
    original_col_names : list
        List of original column names (without suffixes)
    """
    import numpy as np
    import pandas as pd
    
    def clean_string_value(value):
        """Convert to string and strip any leading/trailing spaces"""
        return str(value).strip()
    
    print("Analyse améliorée des colonnes dupliquées:")
    print("-" * 70)
    
    for col in original_col_names:
        baseline_col = f"{col}"
        final_col = f"{col}_final"
        
        if baseline_col in merged_df.columns and final_col in merged_df.columns:
            # Get data types
            baseline_type = merged_df[baseline_col].dtype
            final_type = merged_df[final_col].dtype
            
            print(f"\nColonne: {col}")
            print(f"Type Baseline: {baseline_type}")
            print(f"Type Final: {final_type}")
            
            if pd.api.types.is_numeric_dtype(baseline_type) and pd.api.types.is_numeric_dtype(final_type):
                is_equal = np.isclose(merged_df[baseline_col], merged_df[final_col], 
                                    rtol=1e-05, atol=1e-08, equal_nan=True)
            else:
                baseline_clean = merged_df[baseline_col].astype(str).apply(clean_string_value)
                final_clean = merged_df[final_col].astype(str).apply(clean_string_value)
                is_equal = baseline_clean == final_clean
            
            total_rows = len(merged_df)
            matching_rows = is_equal.sum()
            match_percentage = (matching_rows / total_rows) * 100
            
            print("\nRésultats de la comparaison:")
            print(f"Total des lignes: {total_rows}")
            print(f"Lignes identiques: {matching_rows} ({match_percentage:.2f}%)")
            
            if matching_rows < total_rows:
                print("\nExemples de différences:")
                mismatches = merged_df[~is_equal][[baseline_col, final_col]].head()
                print(mismatches)
                
            print("\n" + "-" * 70)

common_columns = [
    'NEFG2',
    'N_sans_P',
    'clust',
    'final_capillarite_agg',
    'final_fibrin_alv',
    'final_hyperplasie_pn',
    'final_infiltrat_A_A1_A2',
    'final_infiltrat_B_BNSP',
    'final_mPNPorg',
    'final_pn_alv'
]

compare_duplicate_columns(merged_df, common_columns)

Analyse améliorée des colonnes dupliquées:
----------------------------------------------------------------------

Colonne: NEFG2
Type Baseline: object
Type Final: object

Résultats de la comparaison:
Total des lignes: 5038
Lignes identiques: 0 (0.00%)

Exemples de différences:
    NEFG2 NEFG2_final
0  281955         NaN
1  281955         NaN
2  281955         NaN
3  281955         NaN
4  281955         NaN

----------------------------------------------------------------------

Colonne: clust
Type Baseline: float64
Type Final: float64

Résultats de la comparaison:
Total des lignes: 5038
Lignes identiques: 0 (0.00%)

Exemples de différences:
   clust  clust_final
0    3.0          NaN
1    2.0          NaN
2    3.0          NaN
3    1.0          NaN
4    1.0          NaN

----------------------------------------------------------------------

Colonne: final_capillarite_agg
Type Baseline: object
Type Final: object

Résultats de la comparaison:
Total des lignes: 5038
Lignes identiques: 0